In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np

In [102]:
train_df = pd.read_csv('../../results/data/ml/train_data.csv', index_col=0)
test_df = pd.read_csv('../../results/data/ml/test_data.csv', index_col=0)
test2_df = pd.read_csv('../../results/data/ml/test_data_SH.csv', index_col=0)

In [3]:
train_df.group.value_counts()

group
HCC    226
LC     104
CHB    102
HC      70
Name: count, dtype: int64

In [103]:
# HCC vs Control (HC + CHB + LC)

train_data = TabularDataset(train_df)
test_data = TabularDataset(test_df)
test2_data = TabularDataset(test2_df)

train_data[["group"]] = train_data[["group"]] == "HCC"
test_data[["group"]] = test_data[["group"]] == "HCC"
test2_data[["group"]] = test2_data[["group"]] == "HCC"

In [ ]:
glycan_predictor = TabularPredictor('group', eval_metric='roc_auc').fit(glycan_train_data, time_limit=300, presets='best_quality')

In [69]:
glycan_predictor.evaluate(test_data)

{'roc_auc': 0.8665757859759734,
 'accuracy': 0.7935779816513762,
 'balanced_accuracy': 0.7884893925193832,
 'mcc': 0.5805761764934408,
 'f1': 0.7619047619047619,
 'precision': 0.782608695652174,
 'recall': 0.7422680412371134}

In [104]:
glycan_predictor.evaluate(test2_data)

{'roc_auc': 0.8414125904803684,
 'accuracy': 0.7777777777777778,
 'balanced_accuracy': 0.7802149594209257,
 'mcc': 0.5343436339759967,
 'f1': 0.6981132075471698,
 'precision': 0.6271186440677966,
 'recall': 0.7872340425531915}

In [92]:
test2_pred = pd.DataFrame({
    'sample': test2_df.index,
    'group': test2_df.group,
    'true': test2_data.group,
    'pred': glycan_predictor.predict(test2_data),
    'proba': glycan_predictor.predict_proba(test2_data).iloc[:,1]
})
test2_pred[test2_pred.true != test2_pred.pred]

,sample,group,true,pred,proba
sample,,,,,
S2011,S2011,LC,False,True,0.685070
S2006,S2006,CHB,False,True,0.540974
S2009,S2009,LC,False,True,0.720274
S2014,S2014,LC,False,True,0.682106
S2019,S2019,LC,False,True,0.656526
S2020,S2020,HCC,True,False,0.397629
S2026,S2026,CHB,False,True,0.698583
S2028,S2028,LC,False,True,0.698780
S2029,S2029,LC,False,True,0.765417


In [105]:
# Train three models to predict HCC vs HC, HCC vs CHB, HCC vs LC
# using only glycans.

HCC_HC_train_df = train_df[np.isin(train_df.group, ['HCC', 'HC'])]
HCC_HC_test_df = test_df[np.isin(test_df.group, ['HCC', 'HC'])]
HCC_HC_test2_df = test2_df[np.isin(test2_df.group, ['HCC', 'HC'])]

HCC_HC_train_data = TabularDataset(HCC_HC_train_df)
HCC_HC_test_data = TabularDataset(HCC_HC_test_df)
HCC_HC_test2_data = TabularDataset(HCC_HC_test2_df)

HCC_HC_train_data['group'] = HCC_HC_train_data['group'] == 'HCC'
HCC_HC_test_data['group'] = HCC_HC_test_data['group'] == 'HCC'
HCC_HC_test2_data['group'] = HCC_HC_test2_data['group'] == 'HCC'

In [73]:
HCC_HC_predictor = TabularPredictor('group', eval_metric='roc_auc').fit(HCC_HC_train_data, time_limit=300, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20241209_051439"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.8
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.1.0: Thu Oct 10 21:02:26 PDT 2024; root:xnu-11215.41.3~2/RELEASE_ARM64_T8122
CPU Count:          8
Memory Avail:       3.93 GB / 16.00 GB (24.6%)
Disk Space Avail:   26.69 GB / 460.43 GB (5.8%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_sta

In [74]:
HCC_HC_predictor.evaluate(HCC_HC_test_data)

{'roc_auc': 0.9334885267708679,
 'accuracy': 0.84375,
 'balanced_accuracy': 0.7871632856667775,
 'mcc': 0.574326571333555,
 'f1': 0.8969072164948454,
 'precision': 0.8969072164948454,
 'recall': 0.8969072164948454}

In [106]:
HCC_HC_predictor.evaluate(HCC_HC_test2_data)

{'roc_auc': 0.9157119476268413,
 'accuracy': 0.8493150684931506,
 'balanced_accuracy': 0.8228314238952537,
 'mcc': 0.665568071785109,
 'f1': 0.8865979381443299,
 'precision': 0.86,
 'recall': 0.9148936170212766}

In [107]:
# Train three models to predict HCC vs HC, HCC vs CHB, HCC vs LC
# using only glycans.

HCC_CHB_train_df = train_df[np.isin(train_df.group, ['HCC', 'CHB'])]
HCC_CHB_test_df = test_df[np.isin(test_df.group, ['HCC', 'CHB'])]
HCC_CHB_test2_df = test2_df[np.isin(test2_df.group, ['HCC', 'CHB'])]

HCC_CHB_train_data = TabularDataset(HCC_CHB_train_df)
HCC_CHB_test_data = TabularDataset(HCC_CHB_test_df)
HCC_CHB_test2_data = TabularDataset(HCC_CHB_test2_df)

HCC_CHB_train_data['group'] = HCC_CHB_train_data['group'] == 'HCC'
HCC_CHB_test_data['group'] = HCC_CHB_test_data['group'] == 'HCC'
HCC_CHB_test2_data['group'] = HCC_CHB_test2_data['group'] == 'HCC'

In [80]:
HCC_CHB_predictor = TabularPredictor('group', eval_metric='roc_auc').fit(HCC_CHB_train_data, time_limit=300, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20241209_052142"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.8
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.1.0: Thu Oct 10 21:02:26 PDT 2024; root:xnu-11215.41.3~2/RELEASE_ARM64_T8122
CPU Count:          8
Memory Avail:       4.02 GB / 16.00 GB (25.1%)
Disk Space Avail:   27.62 GB / 460.43 GB (6.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_sta

In [81]:
HCC_CHB_predictor.evaluate(HCC_CHB_test_data)

{'roc_auc': 0.8969072164948453,
 'accuracy': 0.8661971830985915,
 'balanced_accuracy': 0.8603665521191295,
 'mcc': 0.7020817408643542,
 'f1': 0.8994708994708994,
 'precision': 0.9239130434782609,
 'recall': 0.8762886597938144}

In [108]:
HCC_CHB_predictor.evaluate(HCC_CHB_test2_data)

{'roc_auc': 0.8244680851063829,
 'accuracy': 0.7469879518072289,
 'balanced_accuracy': 0.734338061465721,
 'mcc': 0.48016498759598053,
 'f1': 0.7878787878787878,
 'precision': 0.75,
 'recall': 0.8297872340425532}

In [109]:
# Train three models to predict HCC vs HC, HCC vs CHB, HCC vs LC
# using only glycans.

HCC_LC_train_df = train_df[np.isin(train_df.group, ['HCC', 'LC'])]
HCC_LC_test_df = test_df[np.isin(test_df.group, ['HCC', 'LC'])]
HCC_LC_test2_df = test2_df[np.isin(test2_df.group, ['HCC', 'LC'])]

HCC_LC_train_data = TabularDataset(HCC_LC_train_df)
HCC_LC_test_data = TabularDataset(HCC_LC_test_df)
HCC_LC_test2_data = TabularDataset(HCC_LC_test2_df)

HCC_LC_train_data['group'] = HCC_LC_train_data['group'] == 'HCC'
HCC_LC_test_data['group'] = HCC_LC_test_data['group'] == 'HCC'
HCC_LC_test2_data['group'] = HCC_LC_test2_data['group'] == 'HCC'

In [85]:
HCC_LC_predictor = TabularPredictor('group', eval_metric='roc_auc').fit(HCC_LC_train_data, time_limit=300, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20241209_060255"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.8
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.1.0: Thu Oct 10 21:02:26 PDT 2024; root:xnu-11215.41.3~2/RELEASE_ARM64_T8122
CPU Count:          8
Memory Avail:       3.87 GB / 16.00 GB (24.2%)
Disk Space Avail:   26.82 GB / 460.43 GB (5.8%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_sta

In [86]:
HCC_LC_predictor.evaluate(HCC_LC_test_data)

{'roc_auc': 0.838717067583047,
 'accuracy': 0.7746478873239436,
 'balanced_accuracy': 0.6742268041237114,
 'mcc': 0.44004763507398714,
 'f1': 0.8518518518518519,
 'precision': 0.773109243697479,
 'recall': 0.9484536082474226}

In [110]:
HCC_LC_predictor.evaluate(HCC_LC_test2_data)

{'roc_auc': 0.762917933130699,
 'accuracy': 0.6585365853658537,
 'balanced_accuracy': 0.6145896656534955,
 'mcc': 0.29320769794136575,
 'f1': 0.7543859649122807,
 'precision': 0.6417910447761194,
 'recall': 0.9148936170212766}